In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

path = r'D:\UniFRK\SEXTANT_sept2018\S-TFMoX\enant=S_hel=+1_KE=3.0eV' # use your path
all_files = glob.glob(pat
cosphi_th = []h + "/*.dat")

li = []
colnames=["phi","cos(theta)","value"] 

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None) 
    # or delim_whitespace=True, it is faste
    # r"\s+" is a regex (regular expression)
    df["filename"]=filename.split("\\")[-1].split(".")[0] # adding a column with the file name
    df["filename"]=df["filename"].str.repl
    cosphi_th.append((int(df["filename"].str.split("_")[0][0]),int(df["filename"].str.split("_")[0][1])))ace(" ","") # corrects for spaces
    df["filename"]=df["filename"].astype("category")
    
cosphi_th=np.array(cosphi_th)li.append(df) # a unique DataFrame

frame = pd.concat(li, axis=0)
frame_srt=frame.groupby("filename")
frame.loc[frame["filename"] == "1_1"] # first way of selecting the right file
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)
counts = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # counts

#Forcing the values at the edges of +-phi to be the same avg
test11=frame.loc[frame["filename"] == "1_1"]
a1=test11.loc[test11["phi"] == np.min(phi)]["value"].to_numpy()
b1=test11.loc[test11["phi"] == np.max(phi)]["value"].to_numpy()
avgphi=np.divide(a1+b1,2)
avgphi=np.pad(avgphi, (0,len(phi)-len(avgphi)), 'constant', constant_values=0)
# avgphipd = pd.Series(avgphi)
test11["value"].mask(test11["phi"] == np.min(phi), avgphi, inplace=True)
test11["value"].mask(test11["phi"] == np.max(phi), avgphi, inplace=True)

#Forcing the values at the edges of +-ctheta to be the same avg
a2=test11.loc[test11["cos(theta)"] == np.min(ctheta)]["value"].to_numpy()
b2=test11.loc[test11["cos(theta)"] == np.max(ctheta)]["value"].to_numpy()
avgcos=np.divide(a2+b2,2)
avgcos=np.pad(avgcos, (0,len(ctheta)-len(avgcos)), 'constant', constant_values=0)
# avgcospd = pd.Series(avgcos)
# test11["value"].mask(test11["cos(theta)"] == np.min(ctheta), avgcos, inplace=True)
# test11["value"].mask(test11["cos(theta)"] == np.max(ctheta), avgcos, inplace=True)

counts_fixed = test11.iloc[:,2].to_numpy()
phi_fixed = np.where(phi==np.min(phi), -180., phi) #the last is an else
phi_fixed = np.where(phi==np.max(phi), 180., phi_fixed)
ctheta_fixed = np.where(ctheta==np.min(ctheta), -1., ctheta)
ctheta_fixed = np.where(ctheta==np.max(ctheta), 1., ctheta_fixed)
# print(np.unique(phi_fixed))

theta_rad = np.arccos(ctheta)
# theta_rad = np.arccos(ctheta_fixed)
phi_rad = phi * np.pi/180.
# phi_rad = phi_fixed * np.pi/180.
countsn = counts
# countsn = counts_fixed

# convertion to spherical coordinates 1D vectors of shape (20000,1)
x = countsn * np.sin(theta_rad) * np.cos(phi_rad)
X = x.reshape(200,100)
y = countsn * np.sin(theta_rad) * np.sin(phi_rad)
Y = y.reshape(200,100)
z = countsn * np.cos(theta_rad)
Z = z.reshape(200,100)

d = np.sqrt(x**2+y**2+z**2)
d_matrix = np.sqrt(X**2+Y**2+Z**2)

In [ ]:
fig=plt.figure()
fig.show()
ax=fig.add_subplot(111)

# ax.plot(counts_fixed[0:200]-counts[0:200],c="b",label='diff_counts[0:200]')
ax.plot(counts_fixed-counts,c="b",label='diff_counts')
ax.plot(a1-b1,c="g",label='diff_+phi_-phi')
ax.plot(a2-b2,c="y",label='diff_+ctheta_-ctheta')
# ax.plot(avgcos[0:200]-counts[0:200],c="r",label='diff_+avgcos_-counts[0:200]')
# ax.plot(avgphi[0:200]-counts[0:200],c="m",label='diff_+avgphi_-counts[0:200]')
# ax.set_yscale('log')
plt.legend(loc="best")
print(np.min(phi_fixed),np.max(phi_fixed),np.min(ctheta_fixed),np.max(ctheta_fixed))

In [ ]:
fig = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z,
                      opacity=0.5,
                      color="rgba(244,22,100,06)"
                 
 )])
fig.update_layout(title="TFMeOx MFPADs theory triangulated",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [ ]:
# Go scatters do not provide a legend
fig = go.Figure(data=[go.Scatter3d(x=ctheta, y=phi, z=counts,
                    mode='markers',
                    # mode='lines',
                    marker=dict(
                    size=1.5,
                    colorscale='Viridis',   # choose a colorscale
                    opacity=0.85,
                    showscale=True          # to show the legend according to the color
                   )
                )])

fig.update_layout(title="TFMeOx MFPADs spherical coordinates",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #   margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
# Go scatters do not provide a legend
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                    mode='markers',
                    marker=dict(
                        size=5,
                        color=d,            # set color to an array/list of desired values
                        colorscale='Viridis',   # choose a colorscale
                        opacity=1.,
                        showscale=True          # to show the legend according to the color
                   )
                )])

fig.update_layout(title="TFMeOx MFPADs theory scattered",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #   margin=dict(l=0, r=0, b=0, t=0))
fig.show()

the Z distribution has the ending and the starting data that aren't matching. This is regardless the reshaping

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z[0:200,0:100], x=X[0:200,0:100], y=Y[0:200,0:100], surfacecolor=d_matrix, connectgaps=True)])
fig.update_layout(title='TFMeOx MFPADs theory surf', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

It is possible to force the equality and therefore bridging the gap

In [ ]:
#fixing the stripe
Y[-1,0:100]=Y[0,0:100]
X[-1,0:100]=X[0,0:100]
Z[-1,0:100]=Z[0,0:100]

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z[0:200,0:100], x=X[0:200,0:100], y=Y[0:200,0:100], surfacecolor=d_matrix, connectgaps=True)])
fig.update_layout(title='TFMeOx MFPADs theory surf', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z[0:200,0:100], surfacecolor=d_matrix, connectgaps=True)])
fig.update_layout(title='TFMeOx MFPADs theory surf', autosize=False,
                #   width=500, height=500,
                #   margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()